# Clustering Projet de consulting

Une grande entreprise de technologie a besoin de votre aide, elle a été piratée! Heureusement, leurs ingénieurs légistes ont récupéré des informations précieuses sur les piratages, y compris des informations telles que le temps de session, les lieux, la vitesse de frappe des mots / minute, etc. L’ingénieur légiste vous donne les données de chaque session utilisées par les pirates pour se connecter à leurs serveurs. Ce sont les caractéristiques des données:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


La société de technologie a 3 pirates potentiels qui ont perpétré l’attaque. Ils sont certains des deux premiers hackers mais ne sont pas très sûrs que le troisième pirate soit impliqué ou non. Ils ont demandé votre aide! Pouvez-vous nous aider à déterminer si le troisième suspect avait un lien avec les attaques ou s’agissait-il uniquement de deux pirates informatiques? Il est probablement impossible de savoir avec certitude, mais peut-être que ce que vous venez d'apprendre sur le clustering peut aider!

**
Un dernier fait important, l’ingénieur légiste sait que les pirates informatiques tranchent leurs attaques. Ce qui signifie qu'ils devraient chacun avoir à peu près le même nombre d'attaques. Par exemple, s'il y a 100 attaques totales, alors dans une situation de 2 hackers, chacun devrait avoir environ 50 hacks, dans une situation de trois pirates, chacun aura environ 33 hacks. L’ingénieur pense que c’est l’élément clé pour résoudre ce problème, mais il ne sait pas comment distinguer ces données non étiquetées en groupes de pirates.**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

from pyspark.ml.clustering import KMeans

# charger les données .
dataset = spark.read.csv("/FileStore/tables/hack_data.csv",header=True,inferSchema=True)

In [3]:
dataset.head()

Out[ 3 ]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location=u'Slovenia', WPM_Typing_Speed=72.37)

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [5]:
dataset.columns

Out[ 5 ]: 
['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
dataset = dataset.drop(dataset['Location'])

In [7]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [8]:
final_data = vec_assembler.transform(dataset)

In [9]:
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed| features|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| 72.37|[8.0,391.09,1.0,2...|
 20.0| 720.99| 0| 3.04| 9.0| 69.08|[20.0,720.99,0.0,...|
 31.0| 356.32| 1| 3.71| 8.0| 70.58|[31.0,356.32,1.0,...|
 2.0| 228.08| 1| 2.48| 8.0| 70.8|[2.0,228.08,1.0,2...|
 20.0| 408.5| 0| 3.57| 8.0| 71.28|[20.0,408.5,0.0,3...|
 1.0| 390.69| 1| 2.79| 9.0| 71.57|[1.0,390.69,1.0,2...|
 18.0| 342.97| 1| 5.1| 7.0| 72.32|[18.0,342.97,1.0,...|
 22.0| 101.61| 1| 3.03| 7.0| 72.03|[22.0,101.61,1.0,...|
 15.0| 275.53| 1| 3.53| 8.0| 70.17|[15.0,275.53,1.0,...|
 12.0| 424.83| 1| 2.53| 8.0| 69.99|[12.0,424.83,1.0,...|
 15.0| 249.09| 1| 3.39| 9.0| 70.77|[15.0,249.09,1.0,...|
 32.0| 242.48| 0| 4.24| 8.0| 67.93|[32.0,242.48,0.0,...|
 23.0| 514.54| 0| 3.18| 8.0| 68.56|[23.0,514.54,0.0,...|
 9.0| 284.77| 0| 3.12| 9.0| 70.82|[9.0,284.77,0.0,3...|
 27.0| 779.25| 1| 2.37| 8.0| 72.73|[27.0,779.25,1.0,...|
 12.0| 307.31| 1| 3.22| 7.0| 67.95|[12.0,307.31,1.0,...|
 21.0| 355.94| 1| 2.0| 7.0| 72.0|[21.0,355.94,1.0,...|
 10.0| 372.65| 0| 3.33| 7.0| 69.19|[10.0,372.65,0.0,...|
 20.0| 347.23| 1| 2.33| 7.0| 70.41|[20.0,347.23,1.0,...|
 22.0| 456.57| 0| 1.52| 8.0| 69.35|[22.0,456.57,0.0,...|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
only showing top 20 rows

In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [12]:
scalerModel = scaler.fit(final_data)

In [13]:
# Normaliser chaque caractéristique .
final_data = scalerModel.transform(final_data)

In [14]:
# apprendre un modèle K-means 
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [15]:
# Évaluez le regroupement en calculant la somme des erreurs carrées définie.
wssse = model.computeCost(final_data)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 434.755073085

In [16]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 3.05623261 2.95754486 1.99757683 3.2079628 4.49941976 3.26738378]
[ 1.26023837 1.31829808 0.99280765 1.36491885 2.5625043 5.26676612]
[ 2.93719177 2.88492202 0. 3.19938371 4.52857793 3.30407351]

In [17]:
model.transform(final_data).select('Features').show()

+--------------------+
 Features|
+--------------------+
[8.0,391.09,1.0,2...|
[20.0,720.99,0.0,...|
[31.0,356.32,1.0,...|
[2.0,228.08,1.0,2...|
[20.0,408.5,0.0,3...|
[1.0,390.69,1.0,2...|
[18.0,342.97,1.0,...|
[22.0,101.61,1.0,...|
[15.0,275.53,1.0,...|
[12.0,424.83,1.0,...|
[15.0,249.09,1.0,...|
[32.0,242.48,0.0,...|
[23.0,514.54,0.0,...|
[9.0,284.77,0.0,3...|
[27.0,779.25,1.0,...|
[12.0,307.31,1.0,...|
[21.0,355.94,1.0,...|
[10.0,372.65,0.0,...|
[20.0,347.23,1.0,...|
[22.0,456.57,0.0,...|
+--------------------+
only showing top 20 rows

In [18]:
model.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 2| 79|
 0| 88|
+----------+-----+

En se basant sur cette phrase 'l’ingénieur légiste sait que les pirates informatiques tranchent leurs attaques. Ce qui signifie qu'ils devraient chacun avoir à peu près le même nombre d'attaques' on peut conclure que seulement deux pirates ont commis l'attaque et ces ceux dont le count est tres proche l'un de l'autre (88 et 79).